In [45]:
import pandas as pd
from apyori import apriori

# GET DATA
model = 'svd'
city = 'Philadelphia'
k=10

preds = pd.read_pickle(f"../data/clean/predictions_{model}_{city}.pkl")
recs = pd.read_pickle(f"../data/clean/recommendations_{model}_{city}_top_{k}.pkl")

In [46]:
# create association rules
results = pd.DataFrame(list(apriori(recs['recommendations'])))
cols = ['items_base', 'items_add', 'confidence', 'lift']
df_associations=results.explode('ordered_statistics')

df_associations[cols] = pd.DataFrame(
    df_associations['ordered_statistics'].to_list(),
    columns = cols
)
df_associations.drop(columns='ordered_statistics', inplace=True)

In [73]:
# filter according to rules: min_supp = 0.1, min_conf = 0.1, min_lift = 0.1, max_len= 2
df_associations_filter = df_associations[
    (df_associations.apply(lambda x: len(x['items_base'])==len(x['items_add']), axis=1)) & 
    (df_associations['support']>=0.1) &
    (df_associations['confidence']>=0.1) &
    (df_associations['lift']>=0.1)
    ].copy(deep=True)

df_associations_filter['X'] = df_associations_filter['items_base'].apply(lambda x: list(x)[0])
df_associations_filter['Y'] = df_associations_filter['items_add'].apply(lambda x: list(x)[0])
df_associations_filter.drop(columns=['items_base', 'items_add'], inplace=True)

In [75]:
# TODO: for each User, List of Y where X in train and Y not in train